# Neural Networks: Explaining Image Classification using LIME

## Set Up - With ImageDataGenerator

https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator

In [ ]:
import pandas as pd

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.datasets import cifar10
from tensorflow.keras import utils
from tensorflow.keras import layers
from tensorflow.keras import models

In [ ]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [ ]:
pd.Series(y_train.reshape(50000)).value_counts()

In [ ]:
x_train.shape

In [ ]:
input_shape = (32, 32, 3) # Size of each image - 32x32 for 3 layers
num_classes = 10

In [ ]:
y_train = utils.to_categorical(y_train, num_classes)
y_test = utils.to_categorical(y_test, num_classes)

In [ ]:
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.2,
    rescale= 1. / 255.)
# compute quantities required for featurewise normalization
# (std, mean, and principal components if ZCA whitening is applied)
datagen.fit(x_train)

In [ ]:
pretrained = tf.keras.applications.vgg19.VGG19(
    weights='imagenet',
    include_top=False, # Allows us to set input shape
    input_shape=input_shape) 
# May download data at this step, shouldn't take long

In [ ]:
model = models.Sequential()
model.add(pretrained)

# freezing layers so they don't get re-trained with your new data
for layer in model.layers:
    layer.trainable=False 

In [ ]:
# adding our own dense layers
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(num_classes, activation='softmax'))

In [ ]:
model.summary()

With this you can now compile and fit your model!

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer="adam",
              metrics=['accuracy'])

In [ ]:
# fits the model on batches with real-time data augmentation:
history = model.fit(
    datagen.flow(x_train, y_train, 
                 batch_size=32, subset='training'),
    validation_data=datagen.flow(x_train, y_train,
                                 batch_size=8, subset='validation'),
    steps_per_epoch=len(x_train) / 32, 
    epochs=5)

In [ ]:
# Evaluate!
score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

# Visualize results
visualize_training_results(history)

## Building Out Model Explainability Using LIME

https://github.com/marcotcr/lime/blob/master/doc/notebooks/Tutorial%20-%20Image%20Classification%20Keras.ipynb